In [ ]:
#r "nuget:OpenCvSharp4"
#r "nuget:OpenCvSharp4.Extensions"
#r "nuget:OpenCvSharp4.runtime.win"

Installed Packages OpenCvSharp4, 4.5.5.20211231 OpenCvSharp4.Extensions, 4.5.5.20211231 OpenCvSharp4.runtime.win, 4.5.5.20211231

In [ ]:
using System;
using System.IO;
using OpenCvSharp;

In [ ]:
public static Mat MakeOverlay(Mat overlayOG, Mat face)
{
    int height = 132;
    int width = 176;
    int x = 114;
    int y = 120;

    using(Mat overlay = new Mat())
    {
        Cv2.Resize(overlayOG, overlay, new Size() { Height = height, Width = width });

        // First we need to pull the Alpha channel out to preserve it before we modify the image, this is channel 4, index 3
        Mat alphaChannel = overlay.ExtractChannel(3);

        // Next we convert the image over to BGR2RGB
        Mat overlyConvertedColors = new Mat();
        Cv2.CvtColor(overlay, overlyConvertedColors, ColorConversionCodes.BGR2RGB);

        // Create a Mat[] to hold the remaining 3 color channels of the image
        Mat[] overlyColorChannels = new Mat[3];

        // We know the image is transparent as has 4 channels, we only need the first 3 which contain the color channels
        for (int i = 0; i <= 2; i++)
        {
            Mat l = overlay.ExtractChannel(i);
            overlyColorChannels[i] = l;
        }

        // We need to merge our converted color channels back into a 3 channel image
        Mat overlyMergedConvertedColors = new Mat();
        Cv2.Merge(overlyColorChannels, overlyMergedConvertedColors);

        // We need to convert our alpha channel to a 3 channel image to match our other images
        Mat overlayAlpha3Channel = new Mat();
        Cv2.Merge(new Mat[] { alphaChannel, alphaChannel, alphaChannel }, overlayAlpha3Channel);

        // We need to combine our color image layer with our alpha image layer
        Mat overlayBlackBackground = new Mat();
        Cv2.BitwiseAnd(overlyMergedConvertedColors, overlayAlpha3Channel, overlayBlackBackground);

        // We need to get the ROI of the face where the hat will go
        Rect r = new Rect();
        r.Height = height;
        r.Width = width;
        r.X = x;
        r.Y = y;

        // Once we have our ROI we pull that into a new Mat
        Mat faceROI = face[r];

        // Perform all the bitwise operations to fill in the appropriate pixels
        Mat bitwiseNotOverlay = new Mat();
        Cv2.BitwiseNot(overlayAlpha3Channel, bitwiseNotOverlay);
        Mat faceROIImage = new Mat();
        Cv2.BitwiseAnd(faceROI, bitwiseNotOverlay, faceROIImage);
        Mat faceROIFinal = new Mat();
        Cv2.BitwiseOr(faceROIImage, overlayBlackBackground, faceROIFinal);

        // Place the ROI back on the image
        face[r] = faceROIFinal;

        return face;
    }
}

In [ ]:
var cascade = new CascadeClassifier(@"../Data/haarcascade_frontalface_alt.xml");
var color = Scalar.FromRgb(0, 255, 0);

using(var srcImage = new Mat("../Images/face.jpg"))
using(var grayImage = new Mat())
using(var overlay = new Mat("../Images/sunglasses.png", ImreadModes.Unchanged))
{
    Cv2.CvtColor(srcImage, grayImage, ColorConversionCodes.BGRA2GRAY);
    Cv2.EqualizeHist(grayImage, grayImage);

    var faces = cascade.DetectMultiScale(
        image: grayImage,
        minSize: new Size(60, 60)
        );

    Console.WriteLine("Detected faces: {0}", faces.Length);

    var overlaySrc = MakeOverlay(overlay, srcImage);

    Cv2.ImShow("Face Detection", overlaySrc);
    int key = Cv2.WaitKey(0);
}

Detected faces: 1
